# Game Genres Notebook

In [79]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as pg
import pandas.io.sql as pd_sql
from copy import deepcopy
import numpy as np
import pickle
from fuzzywuzzy import process

In [2]:
# Postgres info to connect

connection_args = {
 "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
 "user": "postgres",
   "password":"FwwBFmleh65qYxKxDVb9",
 "port": 5432,
 "dbname": "twitchdata"
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [3]:
query = '''
SELECT * FROM game_genres ORDER BY game_name;
'''
df=pd_sql.read_sql(query, connection)

In [4]:
print(df.columns)
df.shape

Index(['game_name', 'game_genres'], dtype='object')


(2173, 2)

In [5]:
df.drop_duplicates(inplace = True)

In [6]:
genres = df['game_genres'].unique()

In [7]:
genres

array(['Rhythm & Music Game', 'Platformer', 'Adventure Game',
       'Simulation', 'Shooter', 'FPS', 'Strategy', 'RTS', 'NONE',
       'Indie Game', 'MMORPG', 'Horror', 'Action', 'Open World',
       'Driving/Racing Game', 'Puzzle', 'RPG', 'Stealth', 'Sports Game',
       'MOBA', 'Creative', 'Card & Board Game', 'Survival', 'IRL',
       "Shoot 'Em Up", 'Roguelike', 'Fighting', 'Compilation',
       'Hidden Objects', 'Series: Souls', 'Visual Novel',
       'Flight Simulator', 'Metroidvania', 'Educational Game',
       'Point and Click', 'RETROGAMEPLACEHOLDER', 'Pinball',
       'Gambling Game'], dtype=object)

In [10]:
df.loc[df['game_genres'] == 'NONE']

,game_name,game_genres
8,Agar.io,NONE
29,Always On,NONE
47,Anime Land,NONE
61,A Plague Tale: Innocence,NONE
64,Arena of Evolution: Red Tides,NONE
117,Atelier Lulua: The Scion of Arland,NONE
126,Auto Chess Mobile,NONE
196,Blackjack,NONE
267,Call of Duty: Modern Warfare Remastered,NONE
354,Crossout,NONE


# Goal:
Given a string first:
- Fuzzy match any titles
    - if title matches, show game and related games
- if string does not fuzzy match any titles
    - find similarity of string to all genres
    - take the top 3 similarities and show them?
    

In [50]:
# query = '''SELECT DISTINCT(game_name) FROM game_genres ORDER BY game_name;'''
# df=pd_sql.read_sql(query, connection)

In [20]:
# df.drop_duplicates(inplace = True)

In [ ]:
x.loc[x['game_genres']].apply(lambda x: type(x) == None)

In [22]:
df.head()

,game_name
0,140
1,2Dark
2,7 Days to Die
3,8-Bit Armies
4,Agar.io


In [90]:
df['game_name'].unique()

array(['140', '2Dark', '7 Days to Die', ..., 'Zombie Army Trilogy',
       'zombsroyale.io', 'Zula'], dtype=object)

In [28]:
token2match = "Shoot 'em Up"
match_genre(token2match,genres)[0][0]

"Shoot 'Em Up"

In [22]:
def match_genre(input_token,genre_list):
    genre_ratios = process.extract(input_token,genre_list,limit=10)
    return genre_ratios

In [16]:
genres= list(genres)

In [14]:


streamer_genres_split
# streamer_games_split = list(streamer_games.split(', '))

['Action', 'Stealth', "Shoot 'em Up "]

In [122]:
streamer_games = 'League, monster wo'
streamer_genres = "Action, Stealth, Shoot 'em Up, spo, other "

In [123]:
parse_user_list(streamer_games, list(games['game_name'].unique()))

['Rocket League', 'Monster Hunter World']

In [121]:
parse_user_list(streamer_genres, list(genres['game_genres'].unique()))

['Action', 'Stealth', 'Sports Game']

In [114]:

def find_fuzzy_matches(input_token,comparison_list):
    match_ratios = process.extract(input_token,comparison_list,limit=10)
    return match_ratios

def parse_user_list(user_input_list,unique_items_list):
    fuzzy_parsed = list()
    user_input_split = list(user_input_list.split(', '))
    for token in user_input_split:
        if token.lower() in [x.lower() for x in unique_items_list]:
            fuzzy_parsed.append(token)
        else:
            first_match = find_fuzzy_matches(token,unique_items_list)
            if first_match[0][1] > 70:
                fuzzy_parsed.append(find_fuzzy_matches(token,unique_items_list)[0][0])
    return fuzzy_parsed

In [94]:
def match_string(str2Match):
    
    strOptions = list(df['game_name'])
    game_ratios = process.extract(str2Match,strOptions,limit=10)
    print("Game ratios: ", game_ratios)
    # You can also select the string with the highest matching percentage
    # highest = process.extractOne(str2Match,strOptions)
    strOptions = list(genres)

    genre_ratios = process.extract(str2Match,strOptions,limit=10)
    print("Genre ratios: ", genre_ratios)
    return game_ratios,genre_ratios

In [96]:
match_string('None')

Game ratios:  [('Albion Online', 68), ('A Plague Tale: Innocence', 68), ('Assetto Corsa Competizione', 68), ('Assetto Corsa Competizione', 68), ('Astroneer', 68), ('Astroneer', 68), ('Bloodborne', 68), ('Bloodborne', 68), ('Bloodborne', 68), ('Clone Drone in the Danger Zone', 68)]
Str2match:  None
Genre ratios:  [('NONE', 100), ('Visual Novel', 68), ('Simulation', 51), ('Action', 51), ('Platformer', 45), ('Shooter', 45), ('Indie Game', 45), ('Open World', 45), ('Compilation', 45), ('Hidden Objects', 45)]


([('Albion Online', 68),
  ('A Plague Tale: Innocence', 68),
  ('Assetto Corsa Competizione', 68),
  ('Assetto Corsa Competizione', 68),
  ('Astroneer', 68),
  ('Astroneer', 68),
  ('Bloodborne', 68),
  ('Bloodborne', 68),
  ('Bloodborne', 68),
  ('Clone Drone in the Danger Zone', 68)],
 [('NONE', 100),
  ('Visual Novel', 68),
  ('Simulation', 51),
  ('Action', 51),
  ('Platformer', 45),
  ('Shooter', 45),
  ('Indie Game', 45),
  ('Open World', 45),
  ('Compilation', 45),
  ('Hidden Objects', 45)])

In [77]:
pwd

'/Users/jchow/Work/MetisCode/twitch-streamer-recommender/gitRoot/Twitch_Streamers_Recommender/Jeremy_Notebooks'

In [91]:
with open( "../Flask/html/static/models/games.pkl", "rb" ) as f:
        games = pickle.load(f)

In [96]:
parse_user_genre_list('World',list(games['game_name'].unique()))

['World of Tanks']

In [95]:
list(games['game_name'].unique())


['Mortal Kombat 11',
 "Tom Clancy's The Division 2",
 'Dead by Daylight',
 'Deathgarden',
 'PUBG MOBILE',
 'Dark Souls',
 'Retro',
 'Dota 2',
 'Path of Exile',
 'Talk Shows & Podcasts',
 'Call of Duty: Black Ops 4',
 'World of Tanks',
 'Arma 3',
 'Outer Wilds',
 'Apex Legends',
 'My Summer Car',
 'Escape From Tarkov',
 'Diablo III: Reaper of Souls',
 "Don't Starve Together",
 'Games + Demos',
 'Days Gone',
 "PLAYERUNKNOWN'S BATTLEGROUNDS",
 'Smite',
 'Black Desert Online',
 'FINAL FANTASY XIV Online',
 'DayZ',
 'Total War: Three Kingdoms',
 'Void Bastards',
 'The Elder Scrolls Online',
 'Battlefield V',
 'Mordhau',
 'Battalion 1944',
 'Rocket League',
 'Sekiro: Shadows Die Twice',
 'World of Warships',
 "Conqueror's Blade",
 'Monster Hunter World',
 'Art',
 'Just Chatting',
 'Realm Royale',
 'Sea of Thieves',
 'Warhammer: Chaosbane',
 'Dark Souls III',
 'Super Mario Maker',
 'Science & Technology',
 'Red Dead Redemption 2',
 'Getting Over It',
 'StarCraft II',
 'Old School RuneScape',


In [80]:
    with open( "../Flask/html/static/models/genres.pkl", "rb" ) as f:
        genres = pickle.load(f)

In [87]:
list(genres['game_genres'].unique())

['Fighting',
 'Open World',
 'RPG',
 'Shooter',
 'Action',
 'Horror',
 'FPS',
 'NONE',
 'RETROGAMEPLACEHOLDER',
 'IRL',
 'MMORPG',
 'MOBA',
 'Simulation',
 'Adventure Game',
 'Driving/Racing Game',
 'Strategy',
 'Indie Game',
 'Roguelike',
 'Compilation',
 'Puzzle',
 'Stealth',
 'Sports Game',
 'Creative',
 'Series: Souls',
 'Platformer',
 'RTS',
 'Card & Board Game',
 'Rhythm & Music Game',
 'Gambling Game',
 'Survival',
 'Metroidvania',
 'Educational Game',
 'Point and Click',
 'Flight Simulator',
 'Hidden Objects',
 'Visual Novel']